In [1]:
from __future__ import print_function
import numpy as np
import sys
import os
import cntk as C

# Paths relative to current python file.
abs_path   = os.path.dirname(os.path.abspath("DAT264x"))
data_path  = os.path.join(abs_path, "data")
model_path = os.path.join(abs_path, "Models")

# Define the reader for both training and evaluation action.
def create_reader(path, is_training, input_dim, label_dim):
    return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
        features=C.io.StreamDef(field='features', shape=input_dim),
        labels=C.io.StreamDef(field='labels',   shape=label_dim)
    )), randomize=is_training, max_sweeps=C.io.INFINITELY_REPEAT if is_training else 1)

# Creates and trains a feedforward classification model for MNIST images
debug_output=False
epoch_size=35000
minibatch_size=64
max_epochs=20
image_height = 64
image_width  = 64
num_channels = 1
input_dim = image_height * image_width * num_channels
num_output_classes = 4

# Input variables denoting the features and label data
input_var = C.ops.input_variable((num_channels, image_height, image_width), np.float32)
label_var = C.ops.input_variable(num_output_classes, np.float32)

# Instantiate the feedforward classification model
scaled_input = C.ops.element_times(C.ops.constant(0.00390625), input_var)

with C.layers.default_options(activation=C.ops.relu, pad=False):
    conv1 = C.layers.Convolution2D((5,5), 32, pad=True)(scaled_input)
    pool1 = C.layers.MaxPooling((3,3), (2,2))(conv1)
    conv2 = C.layers.Convolution2D((3,3), 48)(pool1)
    pool2 = C.layers.MaxPooling((3,3), (2,2))(conv2)
    conv3 = C.layers.Convolution2D((3,3), 64)(pool2)
    f4    = C.layers.Dense(96)(conv3)
    drop4 = C.layers.Dropout(0.5)(f4)
    z     = C.layers.Dense(num_output_classes, activation=None)(drop4)

ce = C.losses.cross_entropy_with_softmax(z, label_var)
pe = C.metrics.classification_error(z, label_var)

reader_train = create_reader(os.path.join(data_path, 'Data-full-35000.txt'), 
                             True, input_dim, num_output_classes)

# Set learning parameters
lr_per_sample    = [0.001]*10 + [0.0005]*10 + [0.0001]
lr_schedule      = C.learning_parameter_schedule_per_sample(lr_per_sample, epoch_size=epoch_size)
mms = [0]*5 + [0.9990239141819757]
mm_schedule      = C.learners.momentum_schedule_per_sample(mms, epoch_size=epoch_size)

# Instantiate the trainer object to drive the model training
learner = C.learners.momentum_sgd(z.parameters, lr_schedule, mm_schedule)
progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=max_epochs)
trainer = C.Trainer(z, (ce, pe), learner, progress_printer)

# define mapping from reader streams to network inputs
input_map = {
    input_var : reader_train.streams.features,
    label_var : reader_train.streams.labels
}

C.logging.log_number_of_parameters(z) ; print()

# Get minibatches of images to train with and perform model training
for epoch in range(max_epochs):       # loop over epochs
    sample_count = 0
    while sample_count < epoch_size:  # loop over minibatches in the epoch
        data = reader_train.next_minibatch(min(minibatch_size, epoch_size - sample_count), input_map=input_map) # fetch minibatch.
        trainer.train_minibatch(data)                                   # update model with it
        sample_count += data[label_var].num_samples                     # count samples processed so far

    trainer.summarize_training_progress()
    z.save(os.path.join(model_path, "ConvNet_MNIST_{}.dnn".format(epoch)))

# Load test data
reader_test = create_reader(os.path.join(data_path, 'Data-test-5000_20180720_070615.txt'), 
                            False, input_dim, num_output_classes)

input_map = {
    input_var : reader_test.streams.features,
    label_var : reader_test.streams.labels
}

# Test data for trained model
epoch_size = 5000
minibatch_size = 250

# process minibatches and evaluate the model
metric_numer    = 0
metric_denom    = 0
sample_count    = 0
minibatch_index = 0

while sample_count < epoch_size:
    current_minibatch = min(minibatch_size, epoch_size - sample_count)

    # Fetch next test min batch.
    data = reader_test.next_minibatch(current_minibatch, input_map=input_map)

    # minibatch data to be trained with
    metric_numer += trainer.test_minibatch(data) * current_minibatch
    metric_denom += current_minibatch

    # Keep track of the number of samples processed so far.
    sample_count += data[label_var].num_samples
    minibatch_index += 1

print("")
print("Final Results: Minibatch[1-{}]: errs = {:0.2f}% * {}".format(minibatch_index+1, (metric_numer*100.0)/metric_denom, metric_denom))
print("")



Training 927636 parameters in 10 parameter tensors.

Learning rate per 1 samples: 0.001
Momentum per 1 samples: 0.0
Finished Epoch[1 of 20]: [Training] loss = 0.346272 * 35000, metric = 13.77% * 35000 12.449s (2811.5 samples/s);
Finished Epoch[2 of 20]: [Training] loss = 0.052865 * 35000, metric = 1.68% * 35000 7.675s (4560.3 samples/s);
Finished Epoch[3 of 20]: [Training] loss = 0.039864 * 35000, metric = 1.18% * 35000 7.662s (4568.0 samples/s);
Finished Epoch[4 of 20]: [Training] loss = 0.026491 * 35000, metric = 0.78% * 35000 7.656s (4571.6 samples/s);
Finished Epoch[5 of 20]: [Training] loss = 0.020456 * 35000, metric = 0.65% * 35000 7.684s (4554.9 samples/s);
Momentum per 1 samples: 0.9990239141819757
Finished Epoch[6 of 20]: [Training] loss = 0.015915 * 35000, metric = 0.44% * 35000 7.675s (4560.3 samples/s);
Finished Epoch[7 of 20]: [Training] loss = 0.014587 * 35000, metric = 0.40% * 35000 7.694s (4549.0 samples/s);
Finished Epoch[8 of 20]: [Training] loss = 0.010048 * 35000, m

In [2]:
print('Metric number: ',metric_numer)
print('Metric denom: ', metric_denom)
print('Error rate: ', metric_numer/metric_denom)

Metric number:  9.0
Metric denom:  5000
Error rate:  0.0018


In [3]:
%%time
out = C.softmax(z)
# Read the data for evaluation
test_file=os.path.join(data_path, "Test-10_20180720_064731.txt")
test_file2=os.path.join(data_path, "Data-submit-20000_20180720_092205.txt")
reader_eval=create_reader(test_file2, False, input_dim, num_output_classes)

x = C.input_variable(input_dim)
y = C.input_variable(num_output_classes)
eval_minibatch_size = 20000
eval_input_map = {x: reader_eval.streams.features, y:reader_eval.streams.labels} 

data = reader_eval.next_minibatch(eval_minibatch_size, input_map=eval_input_map)

img_label = data[y].asarray()
img_data = data[x].asarray()

# reshape img_data to: M x 1 x 64 x 64 to be compatible with model
img_data = np.reshape(img_data, (eval_minibatch_size, 1, 64, 64))

predicted_label_prob = [out.eval(img_data[i]) for i in range(len(img_data))]

# Find the index with the maximum value for both predicted as well as the ground truth
pred = [np.argmax(predicted_label_prob[i]) for i in range(len(predicted_label_prob))]
gtlabel = [np.argmax(img_label[i]) for i in range(len(img_label))]
#print("Label    :", gtlabel[:25])
#print("Predicted:", pred)

Wall time: 21.2 s


In [4]:
count = 0
for i in range(len(pred)):
    if pred[i] != gtlabel[i]:
        count += 1
print("Data: ", len(pred))
print("Count:", count)
print("Error:", count / len(pred))
print("Score:", (len(pred)- count) / len(pred))

Data:  20000
Count: 9
Error: 0.00045
Score: 0.99955


In [6]:
%%time
import csv
count = 0
output = []
with open('data/Test-keras-201807202100.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in reader:
        if count == 0:
            output.append('id,orientation\n')
        if count > 0:
            id = row[0]
            label = pred[count-1]
            output.append('{},{}\n'.format(id, label))
            #print('({0}) file: {1} label: {2}'.format(count, row[0], row[1]) + ' onehot: ' + label)
        count += 1


Wall time: 28 ms


In [7]:
%%time
import time
import datetime
filename = 'data/Submit-CNTK_' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S') + '.csv'
print("Saving", filename )
with open(filename, 'w') as f:
    for row in output:
        f.write(row)
print("File save complete.")

Saving data/Submit-CNTK_20180721_050954.csv
File save complete.
Wall time: 33 ms
